In [4]:
import os
import cv2
import pytesseract
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import joblib
import tensorflow.keras as keras

# Step 1: Load the Text Model
text_model_file = 'text_model.pkl'
text_model = joblib.load(text_model_file)

# Step 2: Load the Nature Model
nature_model_file = 'nature_model300.h5'
nature_model = keras.models.load_model(nature_model_file)

# Step 3: Function to extract text from an image
def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

# Step 4: Function to preprocess the text for prediction
def preprocess_text(text, vectorizer):
    text_features = vectorizer.transform([text])
    return text_features

# Step 5: Function to preprocess the image for nature classification
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

# Step 6: Function to predict the class labels for text and nature
def predict_text_and_nature_classes(image_path, vectorizer):
    text = extract_text_from_image(image_path)
    text_features = preprocess_text(text, vectorizer)
    text_class_label = text_model.predict(text_features)[0]

    img = preprocess_image(image_path)
    nature_class_probabilities = nature_model.predict(img)
    nature_class_label = np.argmax(nature_class_probabilities, axis=-1)[0]

    return text_class_label, nature_class_label

# Load the TfidfVectorizer used during training
vectorizer_file = 'vectorizer.pkl'
vectorizer = joblib.load(vectorizer_file)

# Example usage
image_path = 'what.jpeg'
text_class_label, nature_class_label = predict_text_and_nature_classes(image_path, vectorizer)

# Print the predicted class labels
print("Predicted Text Class Label:", text_class_label)
print("Predicted Nature Class Label:", nature_class_label)

# Class names for text and nature classes (replace with your actual class names)
text_class_names = ['Non islamophobic text','islamophobic text']
nature_class_names = ['Islamophobic image','Non Islamophobic image']

# Function to get the class names for text and nature classes
def get_class_name(class_label, class_names):
    return class_names[class_label]

# Get the class names for the predicted class labels
text_class_name = get_class_name(text_class_label, text_class_names)
nature_class_name = get_class_name(nature_class_label, nature_class_names)

# Print the predicted class names
print("Predicted Text Class:", text_class_name)
print("Predicted Nature Class:", nature_class_name)


1/1 [==============================] - 0s 136ms/step
Predicted Text Class Label: 0
Predicted Nature Class Label: 1
Predicted Text Class: Non islamophobic text
Predicted Nature Class: Non Islamophobic image
